# volatility model using ARCH , GARCH and EWMA #

In [117]:
# import imp lib 

import yfinance as yf 
import numpy as np 
import pandas as pd 



In [143]:
# import data from yfinace 

df=yf.download('JPM',start='2015-01-01', end='2025-01-01')
df=df['Close']
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\2842042311.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download('JPM',start='2015-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Ticker,JPM
Date,
2015-01-02,46.948090
2015-01-05,45.490574
2015-01-06,44.311050
2015-01-07,44.378658
2015-01-08,45.370358
...,...
2024-12-24,238.440521
2024-12-26,239.257263
2024-12-27,237.318710


# ARCH

In [144]:
from arch import arch_model

In [145]:
# calculate daily return

df['returns']=df.pct_change()*100

returns= df['returns'].dropna()
returns

Date
2015-01-05   -3.104526
2015-01-06   -2.592896
2015-01-07    0.152576
2015-01-08    2.234631
2015-01-09   -1.738663
                ...   
2024-12-24    1.644367
2024-12-26    0.342535
2024-12-27   -0.810238
2024-12-30   -0.767091
2024-12-31    0.162962
Name: returns, Length: 2515, dtype: float64

In [146]:
# create and fit ARCH model

model=arch_model(returns, vol='ARCH', p=1)
results=model.fit(disp='off')

# summary 

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Constant Mean - ARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -4637.96
Distribution:                  Normal   AIC:                           9281.91
Method:            Maximum Likelihood   BIC:                           9299.40
                                        No. Observations:                 2515
Date:                Mon, Jul 21 2025   Df Residuals:                     2514
Time:                        19:35:13   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1260  3.305e-02      3.811  1.382e-04 [6.119e-02,  0.191]
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          1.6656      0.121     13.743  5.625e-43 [  1.428,  1.903]
alpha[1]       0.4211  6.281e-02      6.704  2.025e-11 [  0.298,  0.544]
========================================================================

Covariance estimator: robust
"""

analysis 

mu- 0.1260   => avg daily return 

omega =	1.6656  => the long run base level of variance 

alpha[1] = 0.4211  => how much yesterdays squared shock impacts todays variance 

In [147]:
# forecast 5 days ahead 

forecast= results.forecast(horizon=5)
predicted_variance = forecast.variance


# volatility = var sqroot

predicted_volatility= predicted_variance**0.5
predicted_volatility






,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-31,1.290793,1.538572,1.631695,1.669357,1.684965


In [148]:
# calculate avg of predicted volaitilty 

predicted_volatility=[1.479268,	1.567023,	1.577429,	1.578698,	1.578854]
predicted_avg_volatility= sum(predicted_volatility)/len(predicted_volatility)
predicted_avg_volatility

1.5562544

In [149]:
# to compare forecasted volatility with real 

start_date= pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09')



real_df= yf.download('JPM', start_date, end_date)
real_df['retruns']= real_df['Close'].pct_change()*100
real_df = real_df.dropna()

realised_vol = real_df['retruns'].std()*np.sqrt(5)    # since we are predicting for next five day so sqrt (5)
realised_vol

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\2524937218.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df= yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed


1.691952213933136

In [150]:
print("ARCH model pred. volt.", predicted_avg_volatility)
print("acutual volatility", realised_vol)

ARCH model pred. volt. 1.5562544
acutual volatility 1.691952213933136


# GARCH

In [151]:
from arch import arch_model

In [152]:
# create and fit ARCH model

model=arch_model(returns, vol='GARCH', p=1 , q=1)
results=model.fit(disp='off')

# summary 

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -4538.83
Distribution:                  Normal   AIC:                           9085.66
Method:            Maximum Likelihood   BIC:                           9108.98
                                        No. Observations:                 2515
Date:                Mon, Jul 21 2025   Df Residuals:                     2514
Time:                        19:35:30   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1230  2.866e-02      4.293  1.764e-05 [6.686e-02,  0.179]
                              Volatility Model                             
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
omega          0.1765      0.255      0.693      0.488    [ -0.323,  0.676]
alpha[1]       0.1222      0.110      1.111      0.267 [-9.347e-02,  0.338]
beta[1]        0.8080      0.208      3.887  1.017e-04    [  0.401,  1.215]
===========================================================================

Covariance estimator: robust
"""

mu	0.1146; estimated avg daily return 
omega	0.0220 : the long run base level of variance 
alpha[1]	0.0165: how much yesterdays shock affects todays shock
beta[1]	0.9737:  how much yesterdays variance affects todays variance

In [153]:
# forecast 5 days ahead 

forecast= results.forecast(horizon=5)
predicted_variance = forecast.variance


# volatility = var sqroot

predicted_volatility= predicted_variance**0.5
predicted_volatility


,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-31,1.290595,1.313729,1.334889,1.354276,1.372065


In [129]:
# calculate avg of predicted volaitilty 

predicted_volatility=[	1.623484,	1.622253,	1.621033,	1.619824,	1.618627]
predicted_avg_volatility= sum(predicted_volatility)/len(predicted_volatility)
predicted_avg_volatility

1.6210442

In [154]:
# to compare forecasted volatility with real 

start_date= pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09')



real_df= yf.download('JPM', start_date, end_date)
real_df['retruns']= real_df['Close'].pct_change()*100
real_df = real_df.dropna()

realised_vol = real_df['retruns'].std()*np.sqrt(5)
realised_vol

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\1245339672.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df= yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed


1.691952213933136

In [155]:
print("GARCH model pred. volt.", predicted_avg_volatility)
print("acutual volatility", realised_vol)

GARCH model pred. volt. 1.5562544
acutual volatility 1.691952213933136


# EWMA 

In [156]:
#data collection

df = yf.download('JPM', start='2015-01-01', end='2025-01-01')

df = df[['Close']]



df 


C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\2397891173.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('JPM', start='2015-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,JPM
Date,
2015-01-02,46.948090
2015-01-05,45.490574
2015-01-06,44.311050
2015-01-07,44.378658
2015-01-08,45.370358
...,...
2024-12-24,238.440521
2024-12-26,239.257263


In [157]:
# calculate daily return

df['returns']=df.pct_change()

df= df.dropna()

# set lemda value for EWMA model 

lamda= 0.94



In [158]:
# Step 4: Initialize variance and calculate EWMA
ewma_var = []
var_t = df['returns'].var()

for ret in df['returns']:
 variance_tplus1 = lamda*var_t + (1-lamda)* (ret**2)
 ewma_var.append(variance_tplus1)



# Volatility = sqrt(variance)
df['ewma_vol'] = np.sqrt(ewma_var)


# Step 5: Predicted Volatility
latest_daily_vol = df['ewma_vol'].iloc[-1] # predicted volatility
latest_daily_vol


# EWMA can only be used for 1 day prediction


C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\1078368735.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ewma_vol'] = np.sqrt(ewma_var)


0.016721007800705564

In [159]:
# Step 6: Get the data and calculate realized volatility

start_date = pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-03') # buffer for weekends and holiday

real_df = yf.download('JPM', start_date, end_date)
real_df['returns'] = real_df['Close'].pct_change()*100
real_df = real_df.dropna()

realized_vol = real_df['returns']
realized_vol

C:\Users\DELL\AppData\Local\Temp\ipykernel_8028\1437514564.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df = yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Date
2025-01-02    0.120973
Name: returns, dtype: float64

In [160]:
print("GARCH model pred. volt.", latest_daily_vol)
print("acutual volatility", realized_vol)

GARCH model pred. volt. 0.016721007800705564
acutual volatility Date
2025-01-02    0.120973
Name: returns, dtype: float64
